# SQL Instance Permissions

This Notebook will provide you with the permissions granted on an instance at both Instance and Database level. It requires the PowerShell module dbatools and if you wish to create the Excel Workbook then it also requires the ImportExcel PowerShell module

You can check if you have the modules and install them in your user scope with this block

In [4]:
#Check for modules and install
$Modules = 'dbatools', 'importExcel'

if((Get-PsRepository -Name PSGallery).InstallationPolicy -ne 'Trusted'){
    Write-Output "             #######################################################"
    Write-Output "The PowerShell Gallery is not trusted so I will trust it so that we can install the modules without interaction"
    Write-Output "             ######     Please follow the Setting Up notebook ######"
} else {
    Write-Output "The PowerShell Gallery is trusted I will continue"
}
$Modules.ForEach{
    If(-not(Get-Module $psitem -ListAvailable)){
        Write-Output "             #######################################################"
        Write-Output "                        We don't have the $psitem module."
        Write-Output "             ######     Please follow the Setting Up notebook ######"
    }
    else {
        Write-Output "We have the $psitem module already"
    }
}

The PowerShell Gallery is trusted I will continue
We have the dbatools module already
We have the importExcel module already


Run this block below to alter the width of the output pane to avoid the output being truncated

In [1]:
# Run this first to make sure output width does not mess with output - Update output buffer size to prevent clipping in Visual Studio output window.
if( $Host -and $Host.UI -and $Host.UI.RawUI ) {
    $rawUI = $Host.UI.RawUI
    $oldSize = $rawUI.BufferSize
    $typeName = $oldSize.GetType( ).FullName
    $newSize = New-Object $typeName (500, $oldSize.Height)
    $rawUI.BufferSize = $newSize
  }

If you want all of the permissions on one worksheet for a single instance use this code - More options below  

You will need to alter the Directory that you want the Excel file stored in using the variable `$ExcelDirectory` and the SQL instance that you want to get the permissions for with the variable `$SQLInstance`. The file will be named `InstanceName_Permissions_Date.xlsx`

In [1]:
$ExcelDirectory = 'c:\temp\' # Alter this to the directory you want the file created
$SQlinstance = 'localhost,15592'  # Alter this for the SQL Instance you want to get permissions for
$PSDefaultParameterValues = @{
    "*Dba*:SQLCredential" = New-Object System.Management.Automation.PSCredential ((Get-Secret -Name container-demo-sql-user -AsPlainText), (Get-Secret -Name container-demo-sql-pwd))
}
Write-Output "Processing $sqlinstance"

$InstanceName = $SQlinstance.Split('\').Split('.').Split('\').Split(',') -join '_'
$ExcelFile = $ExcelDirectory + '\' + $InstanceName + '_Permssions_OneTab_' + (Get-Date).ToString('yyyy-MM-dd') + '.xlsx'

Write-Output "    FileName is $ExcelFile"

$WorkSheetName = "Permissions"

    $excel = Get-DbaUserPermission -SqlInstance $sqlinstance | Export-Excel -Path $ExcelFile -WorksheetName $WorkSheetName -AutoSize -FreezeTopRow -AutoFilter -PassThru
  
    $rulesparam = @{
        Address   = $excel.Workbook.Worksheets[$WorkSheetName].Dimension.Address
        WorkSheet = $excel.Workbook.Worksheets[$WorkSheetName] 
        RuleType  = 'Expression'      
    }

    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("sysadmin",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("db_owner",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER LOGINS",$E1)))' -BackgroundColor PaleGreen 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER SECURABLES",$E1)))' -BackgroundColor PowderBlue 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB ROLE MEMBERS",$E1)))' -BackgroundColor GoldenRod 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB SECURABLES",$E1)))' -BackgroundColor BurlyWood 

    Close-ExcelPackage $excel


Write-Output ""
Write-Output "Finished Processing $sqlinstance"

$Excel = Open-ExcelPackage -Path $ExcelFile
Add-Worksheet -ExcelPackage $Excel -WorksheetName 'Title' -MoveToStart | Out-Null

$TitleSheet = $excel.Workbook.Worksheets['Title']
$Date = (Get-Date).ToLongDateString()
$TitleSheet.Cells["A1"].value = "This Worksheet shows the User Permissions for each database on $sqlinstance at $Date "
Set-ExcelRange -Worksheet $TitleSheet -Range "A:1" -Bold -FontSize 22 -Underline -UnderLineType Double

$TitleSheet.Cells["B3"].Value = "The Cells are colour coded as follows :-"
Set-ExcelRange -Worksheet $TitleSheet -Range "B3" -Bold -FontSize 18 
$TitleSheet.Cells["E5"].Value = "The Yellow Cells show members of the sysadmin role who have permission to do and access anything on the instance "
$TitleSheet.Cells["E6"].Value = "The Green Cells show the logins on the server"
$TitleSheet.Cells["E7"].Value = "The Blue Cells show the instance level permissions that have been granted to the logins"
$TitleSheet.Cells["E8"].Value = "The Orange Cells show the database role membership for the login"
$TitleSheet.Cells["E9"].Value = "The Brown Cells show specific database permissions that have been granted for the logins"

$TitleSheet.Cells["B11"].Value = "You can filter by Database on the Object column"
Set-ExcelRange -Worksheet $TitleSheet -Range "C11" -FontSize 18

$TitleSheet.Cells["B12"].Value = "You can filter by User/Group/Login on the Member column"
Set-ExcelRange -Worksheet $TitleSheet -Range "C12" -FontSize 18

Set-ExcelRange -Worksheet $TitleSheet -Range  "C5" -BackgroundColor Yellow
Set-ExcelRange -Worksheet $TitleSheet -Range  "C6" -BackgroundColor PaleGreen
Set-ExcelRange -Worksheet $TitleSheet -Range  "C7" -BackgroundColor PowderBlue 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C8" -BackgroundColor GoldenRod 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C9" -BackgroundColor BurlyWood 

Close-ExcelPackage $excel


Write-Output "                ###############                  "
Write-Output "        FileName is $ExcelFile            "
Write-Output "                ###############                  "
Write-Output ""

Invoke-Item $ExcelFile

Processing localhost,15592
    FileName is c:\temp\\localhost_15592_Permssions_OneTab_2020-09-06.xlsx

Finished Processing localhost,15592
                ###############                  
        FileName is c:\temp\\localhost_15592_Permssions_OneTab_2020-09-06.xlsx            
                ###############                  



If you want all of the permissions for a number of instances use this code - More options below  

You will need to alter the Directory that you want the Excel file stored in using the variable `$ExcelDirectory` and the SQL instances that you want to get the permissions for with the variable `$SQLInstances` You will need to add them as `'FULLINSTANCENAME,PORT','FULLINSTANCENAME2,PORT'` or if you have variables set up like `$ProdInstances` you can use that. Each file will be named `InstanceName_Permissions_Date.xlsx`

In [ ]:
$ExcelDirectory = 'c:\temp\' # Alter this to the directory you want the file created
$SQlinstances = 'SQL2017N5' # Alter this for the SQL Instances you want to get permissions for

foreach($sqlinstance in $SQlinstances){
Write-Output "Processing $sqlinstance"

$InstanceName = $SQlinstance.Split('\').Split('.').Split('\').Split(',') -join '_'
$ExcelFile = $ExcelDirectory + '\' + $InstanceName + '_Permssions_OneTab_' + (Get-Date).ToString('yyyy-MM-dd') + '.xlsx'

Write-Output "    FileName is $ExcelFile"

$WorkSheetName = "Permissions"

    $excel = Get-DbaUserPermission -SqlInstance $sqlinstance | Export-Excel -Path $ExcelFile -WorksheetName $WorkSheetName -AutoSize -FreezeTopRow -AutoFilter -PassThru
  
    $rulesparam = @{
        Address   = $excel.Workbook.Worksheets[$WorkSheetName].Dimension.Address
        WorkSheet = $excel.Workbook.Worksheets[$WorkSheetName] 
        RuleType  = 'Expression'      
    }

    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("sysadmin",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("db_owner",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER LOGINS",$E1)))' -BackgroundColor PaleGreen 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER SECURABLES",$E1)))' -BackgroundColor PowderBlue 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB ROLE MEMBERS",$E1)))' -BackgroundColor GoldenRod 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB SECURABLES",$E1)))' -BackgroundColor BurlyWood 

    Close-ExcelPackage $excel


Write-Output ""
Write-Output "Finished Processing $sqlinstance"

$Excel = Open-ExcelPackage -Path $ExcelFile
Add-Worksheet -ExcelPackage $Excel -WorksheetName 'Title' -MoveToStart | Out-Null

$TitleSheet = $excel.Workbook.Worksheets['Title']
$Date = (Get-Date).ToLongDateString()
$TitleSheet.Cells["A1"].value = "This Worksheet shows the User Permissions for each database on $sqlinstance at $Date "
Set-ExcelRange -Worksheet $TitleSheet -Range "A:1" -Bold -FontSize 22 -Underline -UnderLineType Double

$TitleSheet.Cells["B3"].Value = "The Cells are colour coded as follows :-"
Set-ExcelRange -Worksheet $TitleSheet -Range "B3" -Bold -FontSize 18 
$TitleSheet.Cells["E5"].Value = "The Yellow Cells show members of the sysadmin role who have permission to do and access anything on the instance "
$TitleSheet.Cells["E6"].Value = "The Green Cells show the logins on the server"
$TitleSheet.Cells["E7"].Value = "The Blue Cells show the instance level permissions that have been granted to the logins"
$TitleSheet.Cells["E8"].Value = "The Orange Cells show the database role membership for the login"
$TitleSheet.Cells["E9"].Value = "The Brown Cells show specific database permissions that have been granted for the logins"

$TitleSheet.Cells["B11"].Value = "You can filter by Database on the Object column"
Set-ExcelRange -Worksheet $TitleSheet -Range "C11" -FontSize 18

$TitleSheet.Cells["B12"].Value = "You can filter by User/Group/Login on the Member column"
Set-ExcelRange -Worksheet $TitleSheet -Range "C12" -FontSize 18

Set-ExcelRange -Worksheet $TitleSheet -Range  "C5" -BackgroundColor Yellow
Set-ExcelRange -Worksheet $TitleSheet -Range  "C6" -BackgroundColor PaleGreen
Set-ExcelRange -Worksheet $TitleSheet -Range  "C7" -BackgroundColor PowderBlue 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C8" -BackgroundColor GoldenRod 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C9" -BackgroundColor BurlyWood 

Close-ExcelPackage $excel


Write-Output "                ###############                  "
Write-Output "        FileName is $ExcelFile            "
Write-Output "                ###############                  "
Write-Output ""

Invoke-Item $ExcelFile
}

If you just want the results for a single database in the notebook use the next block. If you want to create a colour coded Excel Workbook showing all of the permissions for all of the databases on a seperate worksheet then use this block. You will need to alter the Directory that you want the Excel file stored in usign the variable `$ExcelDirectory` and the SQL instance that you want to get the permissions for with the variable `$SQLInstance`. The file will be named `InstanceName_Permissions_Date.xlsx`

In [ ]:
$ExcelDirectory = 'c:\temp\' # Alter this to the directory you want the file created
$SQlinstance = '' # Alter this for the SQL Instance you want to get permissions for
Write-Output "Processing $sqlinstance"

$InstanceName = $SQlinstance.Split('\').Split('.').Split('\').Split(',') -join '_'
$ExcelFile = $ExcelDirectory + '\' + $InstanceName + '_Permssions_' + (Get-Date).ToString('yyyy-MM-dd') + '.xlsx'

Write-Output "    FileName is $ExcelFile"

$Databases = Get-DbaDatabase -SqlInstance $sqlinstance

foreach ($Database in $Databases) {
    $DatabaseName = $Database.Name
    Write-Output "       Processing $DatabaseName"
    $WorkSheetName = "$($Database.Name) Perms"

    $excel = Get-DbaUserPermission -SqlInstance $sqlinstance -Database $DatabaseName | Export-Excel -Path $ExcelFile -WorksheetName $WorkSheetName -AutoSize -FreezeTopRow -AutoFilter -PassThru
  
    $rulesparam = @{
        Address   = $excel.Workbook.Worksheets[$WorkSheetName].Dimension.Address
        WorkSheet = $excel.Workbook.Worksheets[$WorkSheetName] 
        RuleType  = 'Expression'      
    }

    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("sysadmin",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("db_owner",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER LOGINS",$E1)))' -BackgroundColor PaleGreen 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER SECURABLES",$E1)))' -BackgroundColor PowderBlue 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB ROLE MEMBERS",$E1)))' -BackgroundColor GoldenRod 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB SECURABLES",$E1)))' -BackgroundColor BurlyWood 

    Close-ExcelPackage $excel
    Write-Output "       Finished Processing $($Database.Name)"
}

Write-Output ""
Write-Output "Finished Processing $sqlinstance"

$Excel = Open-ExcelPackage -Path $ExcelFile
Add-Worksheet -ExcelPackage $Excel -WorksheetName 'Title' -MoveToStart | Out-Null

$TitleSheet = $excel.Workbook.Worksheets['Title']
$Date = (Get-Date).ToLongDateString()
$TitleSheet.Cells["A1"].value = "This Worksheet shows the User Permissions for each database on $sqlinstance at $Date "
Set-ExcelRange -Worksheet $TitleSheet -Range "A:1" -Bold -FontSize 22 -Underline -UnderLineType Double

$TitleSheet.Cells["B3"].Value = "The Cells are colour coded as follows"
$TitleSheet.Cells["D4"].Value = "The Yellow Cells show members of the sysadmin role who have permission to do and access anything on the instance "
$TitleSheet.Cells["D5"].Value = "The Green Cells show the logins on the server"
$TitleSheet.Cells["D6"].Value = "The Blue Cells show the instance level permissions that have been granted to the logins"
$TitleSheet.Cells["D7"].Value = "The Orange Cells show the database role membership for the login"
$TitleSheet.Cells["D8"].Value = "The Brown Cells show specific database permissions that have been granted for the logins"

Set-ExcelRange -Worksheet $TitleSheet -Range  "C4" -BackgroundColor Yellow
Set-ExcelRange -Worksheet $TitleSheet -Range  "C5" -BackgroundColor PaleGreen
Set-ExcelRange -Worksheet $TitleSheet -Range  "C6" -BackgroundColor PowderBlue 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C7" -BackgroundColor GoldenRod 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C8" -BackgroundColor BurlyWood 

Close-ExcelPackage $excel


Write-Output "                ###############                  "
Write-Output "        FileName is $ExcelFile            "
Write-Output "                ###############                  "
Write-Output ""

Invoke-Item $ExcelFile

You can get the details for a single database and display in the notebook using the block below. You will need to specify the instance and the database in the code

In [ ]:
Get-DbaUserPermission -SqlInstance 'CEAWD06941.EMEA.ZURICH.CORP,1282' -Database tempdb | Format-Table

## Managed instance needs to be done slightly differently

This code works but prompts for a credential which wont work in ADS right now.

 **HIGHLIGHT the code in the block and press F8 to run it in the terminal where you will be prompted for a Username/ Password**

In [ ]:
$ExcelDirectory = 'c:\temp\' # Alter this to the directory you want the file created
$SQlinstance = '.database.windows.net' # Alter this for the SQL Instance you want to get permissions for
Write-Output "Processing $sqlinstance"

$InstanceName = $SQlinstance.Split('\').Split('.').Split('\').Split(',') -join '_'
$ExcelFile = $ExcelDirectory + '\' + $InstanceName + '_Permssions_' + (Get-Date).ToString('yyyy-MM-dd') + '.xlsx'

Write-Output "    FileName is $ExcelFile"

$SQlCredential = Get-Credential

$inst = Connect-DbaInstance  -SqlInstance $sqlinstance -SqlCredential $SQlCredential

$Databases = $inst.Databases

foreach ($Database in $Databases) {
    $DatabaseName = $Database.Name
    Write-Output "       Processing $DatabaseName"
    $WorkSheetName = "$($Database.Name) Perms"

    $excel = Get-DbaUserPermission -SqlInstance $inst -Database $DatabaseName | Export-Excel -Path $ExcelFile -WorksheetName $WorkSheetName -AutoSize -FreezeTopRow -AutoFilter -PassThru
  
    $rulesparam = @{
        Address   = $excel.Workbook.Worksheets[$WorkSheetName].Dimension.Address
        WorkSheet = $excel.Workbook.Worksheets[$WorkSheetName] 
        RuleType  = 'Expression'      
    }

    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("sysadmin",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("db_owner",$G1)))' -BackgroundColor Yellow -StopIfTrue
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER LOGINS",$E1)))' -BackgroundColor PaleGreen 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("SERVER SECURABLES",$E1)))' -BackgroundColor PowderBlue 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB ROLE MEMBERS",$E1)))' -BackgroundColor GoldenRod 
    Add-ConditionalFormatting @rulesparam -ConditionValue 'NOT(ISERROR(FIND("DB SECURABLES",$E1)))' -BackgroundColor BurlyWood 

    Close-ExcelPackage $excel
    Write-Output "       Finished Processing $($Database.Name)"
}

Write-Output ""
Write-Output "Finished Processing $sqlinstance"

$Excel = Open-ExcelPackage -Path $ExcelFile
Add-Worksheet -ExcelPackage $Excel -WorksheetName 'Title' -MoveToStart | Out-Null

$TitleSheet = $excel.Workbook.Worksheets['Title']
$Date = (Get-Date).ToLongDateString()
$TitleSheet.Cells["A1"].value = "This Worksheet shows the User Permissions for each database on $sqlinstance at $Date "
Set-ExcelRange -Worksheet $TitleSheet -Range "A:1" -Bold -FontSize 22 -Underline -UnderLineType Double

$TitleSheet.Cells["B3"].Value = "The Cells are colour coded as follows"
$TitleSheet.Cells["D4"].Value = "The Yellow Cells show members of the sysadmin role who have permission to do and access anything on the instance "
$TitleSheet.Cells["D5"].Value = "The Green Cells show the logins on the server"
$TitleSheet.Cells["D6"].Value = "The Blue Cells show the instance level permissions that have been granted to the logins"
$TitleSheet.Cells["D7"].Value = "The Orange Cells show the database role membership for the login"
$TitleSheet.Cells["D8"].Value = "The Brown Cells show specific database permissions that have been granted for the logins"

Set-ExcelRange -Worksheet $TitleSheet -Range  "C4" -BackgroundColor Yellow
Set-ExcelRange -Worksheet $TitleSheet -Range  "C5" -BackgroundColor PaleGreen
Set-ExcelRange -Worksheet $TitleSheet -Range  "C6" -BackgroundColor PowderBlue 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C7" -BackgroundColor GoldenRod 
Set-ExcelRange -Worksheet $TitleSheet -Range  "C8" -BackgroundColor BurlyWood 

Close-ExcelPackage $excel


Write-Output "                ###############                  "
Write-Output "        FileName is $ExcelFile            "
Write-Output "                ###############                  "
Write-Output ""

Invoke-Item $ExcelFile